## 1. Instantiate the client

This cell sets up the OCI OpenAI client. Load your sandbox.yaml, pick a model (experiment with different ones!), and initialize the LLM.

# Single-step tool calling: Manual first, then Automatic (Agent)

This notebook demonstrates manual tool calling first, followed by automatic (agent) calling, for a single tool. Designed for students to experiment with basic function calling.

## What this notebook does
Demonstrates manual function calling with a custom tool using OCI Generative AI for LLM, followed by automatic agent orchestration.

## Documentation to reference
- OCI Gen AI: https://docs.oracle.com/en-us/iaas/Content/generative-ai/pretrained-models.htm
- LangChain: https://docs.langchain.com/oss/python/langchain/agents
- How to build tools: https://python.langchain.com/docs/how_to/custom_tools/
- OCI OpenAI compatible SDK: https://github.com/oracle-samples/oci-openai  note: supports OpenAI, XAI & Meta models. Also supports OpenAI Responses API 
- OCI langchain SDK: https://github.com/oracle-devrel/langchain-oci-genai  note: as of Nov 2025 it is not compatible with langchain v1.0. supports all OCI models including Cohere
- OCI GenAI SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/generative_ai_inference/models

## Relevant slack channels
- #generative-ai-users: *for questions on OCI Gen AI* 
- #igiu-innovation-lab: *general discussions on your project* 
- #igiu-ai-learning: *help with sandbox environment or help with running this code*
   

## Env setup
- sandbox.yaml: Contains OCI config, compartment, DB details, and wallet path.
- .env: Load environment variables (e.g., API keys if needed).
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`
#####

## How to run the notebook
Run the cells in order. Make sure your sandbox.yaml is configured. You can also run equivalent Python files like langChain/function_calling/langchain_step.py or langChain/function_calling/langchain_step_manual.py using `uv run <path>`.

## Comments to important sections of notebook
- Cell 1: Instantiate the client and load config
- Cell 2: Define the tool
- Manual section: Demonstrates manual single tool calling
- Automatic section: Shows agent-based single-step tool calling

## Experimentation Tips
- Try changing the LLM_MODEL to different values like 'meta.llama-3.1-405b-instruct' or 'xai.grok-4' to see how responses vary.
- Experiment with the system_prompt in the agent: modify it to change the personality or add more instructions.
- Add more tools to the list and see how the agent chooses which one to use.
- Try different queries to see if the model calls the tool or not.

In [ ]:
import os, sys
from dotenv import load_dotenv
from envyaml import EnvYAML
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent

from langChain.oci_openai_helper import OCIOpenAIHelper


def load_config(config_path):
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None

def pretty_print(response):
    for i, m in enumerate(response["messages"], 1):
        role = getattr(m, "type", m.__class__.__name__)
        content = m.content if isinstance(m.content, str) else str(m.content)
        print(f"{i:>2}. [{role.upper()}] {content}")

load_dotenv()
SANDBOX_CONFIG_FILE = "sandbox.yaml"
LLM_MODEL = "openai.gpt-5"

scfg = load_config(SANDBOX_CONFIG_FILE)
llm = OCIOpenAIHelper.get_langchain_openai_client(
    model_name=LLM_MODEL,
    config=scfg,
    use_responses_api=True,
    reasoning={"effort": "low", "summary": "auto"}
)


## 2. Define the tool

Define a simple weather tool. This is a LangChain tool that the model can call. Try adding parameters or modifying the return value!

In [ ]:
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Gets the weather for a given city"""
    return f"The weather in {city} is 70 Fahrenheit"

tools = [get_weather]
tool_map = {t.name.lower(): t for t in tools}


## Manual single-tool calling

This shows manual tool calling: bind the tool to the model, invoke once to get tool calls, execute the tool, and invoke again with the result. Simple for single-step!

In [ ]:
messages = [HumanMessage("How is the weather in San Francisco?")]
llm_with_tools = llm.bind_tools([get_weather])

ai_message = llm_with_tools.invoke(messages)
print("AI response content:", ai_message.content)
print("Tool calls:", getattr(ai_message, "tool_calls", None))

if getattr(ai_message, "tool_calls", None):
    messages.append(ai_message)
    for tool_call in ai_message.tool_calls:
        tool_name = tool_call["name"].lower()
        selected_tool = tool_map.get(tool_name)
        if not selected_tool:
            print(f"Unknown tool requested: {tool_name}")
            continue
        print(f"Executing tool: {tool_name} with args: {tool_call.get('args', {})}")
        tool_msg = selected_tool.invoke(tool_call)
        print("Tool result:", tool_msg.content)
        messages.append(tool_msg)

final_response = llm_with_tools.invoke(messages)
print("\nFinal model message:")
print(final_response.content)


## Automatic (Agent) single-step

Using LangChain's agent to handle everything automatically. The agent invokes the model, calls tools as needed, and responds. Notice the pirate personality from the system prompt!

In [ ]:
agent = create_agent(
    llm,
    tools=[get_weather],
    system_prompt="talk like a pirate who tells bad dad jokes"
)

messages_agent = [HumanMessage(content="What's the weather in San Francisco?")]

print("Agent single-step invocation:")
response = agent.invoke({"messages": messages_agent})
pretty_print(response)

print("\nAgent single-step stream:")
for chunk in agent.invoke({"messages": messages_agent}, stream_mode="updates"):
    for step, data in chunk.items():
        print(f"step: {step}", flush=True)
        print(f"content: {data['messages'][-1].content_blocks}", flush=True)


## Exercises

1. **Add a tool**: Create a second tool, like `get_population(city: str) -> int`, and update both manual and agent sections to use it. Test with queries like "What's the weather and population in San Francisco?"

2. **Change the system prompt**: Try different prompts, e.g., "reply as a Shakespearean actor" or "be very formal and precise". How does it change the response style?

3. **Model comparison**: Switch the LLM_MODEL to 'xai.grok-4' or 'meta.llama-3.1-405b-instruct' and run the notebook. Note any differences in tool usage or responses.

4. **No tool calls**: Modify the query to something that doesn't require a tool, like "Hello, how are you?" See if the agent still tries to call tools.

5. **Tool errors**: Add a print statement or error in the tool function, e.g., if city is invalid. Observe how the manual and agent handle it.

6. **Streaming details**: Analyze the streaming output closely. What does each 'step' represent, and how does it differ from the non-streaming version?